In [6]:
##################trining the dataset#######################

In [9]:
#import libraries
from __future__ import unicode_literals
import spacy
import random
import spacy 
from spacy import displacy
from spacy.displacy import render
import pandas as pd
from io import StringIO
from tabulate import tabulate
import plotly.plotly as py
import plotly.figure_factory as ff
import json 
from train_dataset import dataset


In [10]:
def train_spacy(dataset,iterations):
    TRAIN_DATA = dataset
    nlp = spacy.blank('en') 
    # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.25,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [11]:
prdnlp = train_spacy(dataset(), 50)

Statring iteration 0


KeyboardInterrupt: 

In [ ]:
# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)


In [ ]:
############################################testing the trained model#####################################


In [13]:
#input text 
text = """Fycompa 4 mg film-coated tabletsfrom __future__ import unicode_literals
import spacy 
from spacy import displacy
from spacy.displacy import render
import pandas as pd
from io import StringIO
from tabulate import tabulate
import plotly.plotly as py
import plotly.figure_factory as ff
crocine 12 mg film-coated tablets 
Quadrameterer 1.3 GBq/mL solution for injection 
Topotecan123 Actavis 1mg powder for concentrate for solution 
4mg –packs of 7, 28, 84 and 98
Topotecan2344 Actavis 2mg powder for concentrate for solution
Topotecan45 Actavis 1mg powder for concentrate for solution
"""

In [ ]:
#loading the trained model 

In [14]:
nlp = spacy.load("cogna")
doc = nlp(text)

In [ ]:
#viusalizing the result 
options = {"ents": ["PACKAGE_ITEM_QTY","Tablet","injection"], "colors": {"PACKAGE_ITEM_QTY":"Red","Tablet":"Yellow"}}
displacy.serve(doc, style="ent")


C:\ProgramData\Anaconda3\lib\runpy.py:193: UserWarning:

[W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.




Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



In [21]:
# data output in form of json 
results=displacy.parse_ents(doc, options=options)
print(json.dumps(results))

{"text": "Fycompa 4 mg film-coated tabletsfrom __future__ import unicode_literals\nimport spacy \nfrom spacy import displacy\nfrom spacy.displacy import render\nimport pandas as pd\nfrom io import StringIO\nfrom tabulate import tabulate\nimport plotly.plotly as py\nimport plotly.figure_factory as ff\ncrocine 12 mg film-coated tablets \nQuadrameterer 1.3 GBq/mL solution for injection \nTopotecan123 Actavis 1mg powder for concentrate for solution \n4mg \u2013packs of 7, 28, 84 and 98\nTopotecan2344 Actavis 2mg powder for concentrate for solution\nTopotecan45 Actavis 1mg powder for concentrate for solution\n", "ents": [{"start": 0, "end": 7, "label": "tablet"}, {"start": 318, "end": 331, "label": "injection"}, {"start": 367, "end": 379, "label": "solution"}, {"start": 457, "end": 459, "label": "package_item_qty"}, {"start": 460, "end": 473, "label": "solution"}, {"start": 522, "end": 533, "label": "solution"}], "title": null, "settings": {"lang": "en", "direction": "ltr"}}


In [22]:
#NER output data as dataframe(tabular format)
param = [[ent.text,ent.label_] for ent in doc.ents]
df=pd.DataFrame(param)
headers = ['Entity','Category']
df.columns = headers 
df_table= pd.read_table(StringIO(str(df)), sep="\s+", header=0)

print(tabulate(df_table, headers='keys', tablefmt='psql'))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning:

read_table is deprecated, use read_csv instead.



+----+---------------+------------------+
|    | Entity        | Category         |
|----+---------------+------------------|
|  0 | Fycompa       | tablet           |
|  1 | Quadrameterer | injection        |
|  2 | Topotecan123  | solution         |
|  3 | 98            | package_item_qty |
|  4 | Topotecan2344 | solution         |
|  5 | Topotecan45   | solution         |
+----+---------------+------------------+


In [5]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

examples = [
    ('4mg –packs of 7, 28, 84 and 98',
     [(29, 31, 'package_item_qty')]),
    ('Quadramet32 1.3 GBq/mL solution for injection',
     [(0, 11, 'injection')])
]

ner_model = spacy.load("cogna") # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, examples)
print(results)

### in dataframe

import pandas as pd 
df = pd.DataFrame(results.items(), columns=["measure", "acuuracy"]) 
print(df)

for name, age in results.items():
    print('{} {}'.format(name, age))

{'uas': 0.0, 'las': 0.0, 'ents_p': 100.0, 'ents_r': 100.0, 'ents_f': 100.0, 'tags_acc': 0.0, 'token_acc': 100.0}
     measure  acuuracy
0        uas       0.0
1        las       0.0
2     ents_p     100.0
3     ents_r     100.0
4     ents_f     100.0
5   tags_acc       0.0
6  token_acc     100.0
uas 0.0
las 0.0
ents_p 100.0
ents_r 100.0
ents_f 100.0
tags_acc 0.0
token_acc 100.0
